In [20]:
import pandas as pd

# 讀取 CSV 檔案
df = pd.read_csv('data_without_2021.csv')


# 確保 datacreationdate 為 datetime 格式
df['datacreationdate'] = pd.to_datetime(df['datacreationdate'])

# 縣市和站點的對應關係
locations = {
    "苗栗": [25, 26, 27],
    "南投": [36, 69, 72],
    "彰化": [33, 34, 35, 83],
    "台中": [28, 29, 30, 31, 32],
    "雲林": [37, 38, 41]
}

# 提取特定站點的資料
specific_siteids = [siteid for siteids in locations.values() for siteid in siteids]
filtered_df = df[df['siteid'].isin(specific_siteids)].copy()

In [13]:
filtered_df.shape

(313083, 17)

In [22]:
# 獲取所有站點的列表
site_ids = filtered_df['siteid'].unique()

# 定義一個字典來存放每個站點缺失的時間點
original_missing_data_summary = {}
# 定義一個字典來存放每個站點缺失的時間點數量
missing_data_count_summary = {}

# 針對每個站點單獨處理，找出缺失的時間點並保存
for site_id in site_ids:
    site_df = filtered_df[filtered_df['siteid'] == site_id].copy()
    site_df = site_df.sort_values(by='datacreationdate')

    # 定義時間範圍
    start_date = site_df['datacreationdate'].min()
    end_date = site_df['datacreationdate'].max()
    expected_dates = pd.date_range(start=start_date, end=end_date, freq='h')

    # 找出缺失的時間點
    missing_dates = expected_dates[~expected_dates.isin(site_df['datacreationdate'])]

    # 保存每個站點缺失的時間點
    original_missing_data_summary[site_id] = missing_dates

# 顯示每個站點原本缺失的時間點
for site_id, missing_dates in original_missing_data_summary.items():
    if not missing_dates.empty:
        print(f"原本站點 {site_id} 缺失的時間點：")
        print(missing_dates)
    else:
        print(f"原本站點 {site_id} 沒有缺失的時間點。")

# 開始填補缺失的時間點
final_df = pd.DataFrame()
for site_id in site_ids:
    site_df = filtered_df[filtered_df['siteid'] == site_id].copy()
    site_df = site_df.sort_values(by='datacreationdate')

    # 定義時間範圍
    start_date = site_df['datacreationdate'].min()
    end_date = site_df['datacreationdate'].max()
    expected_dates = pd.date_range(start=start_date, end=end_date, freq='h')

    # 找出缺失的時間點
    missing_dates = expected_dates[~expected_dates.isin(site_df['datacreationdate'])]
    # 保存每個站點缺失的時間點數量
    missing_data_count_summary[site_id] = len(missing_dates)

    # 複製前一個小時的資料來填補缺失的部分
    for missing_date in missing_dates:
        prev_hour = missing_date - pd.Timedelta(hours=1)
        if prev_hour in site_df['datacreationdate'].values:
            prev_data = site_df[site_df['datacreationdate'] == prev_hour]
            new_data = prev_data.copy()
            new_data['datacreationdate'] = missing_date
            site_df = pd.concat([site_df, new_data]).reset_index(drop=True)

    # 重新排序 DataFrame 並添加到最終結果中
    site_df = site_df.sort_values(by='datacreationdate').reset_index(drop=True)
    final_df = pd.concat([final_df, site_df]).reset_index(drop=True)

原本站點 25.0 缺失的時間點：
DatetimeIndex(['2022-01-24 02:00:00', '2022-01-24 04:00:00',
               '2022-01-24 07:00:00', '2022-01-24 08:00:00',
               '2022-01-24 10:00:00', '2022-01-25 10:00:00',
               '2022-01-25 11:00:00', '2022-01-26 14:00:00',
               '2022-01-26 15:00:00', '2022-01-26 16:00:00',
               ...
               '2023-10-26 00:00:00', '2023-10-26 01:00:00',
               '2023-10-26 02:00:00', '2023-10-26 03:00:00',
               '2023-10-26 04:00:00', '2023-10-26 05:00:00',
               '2023-10-26 06:00:00', '2023-10-26 07:00:00',
               '2023-10-26 08:00:00', '2023-10-26 09:00:00'],
              dtype='datetime64[ns]', length=125, freq=None)
原本站點 26.0 缺失的時間點：
DatetimeIndex(['2022-01-24 02:00:00', '2022-01-24 04:00:00',
               '2022-01-24 07:00:00', '2022-01-24 08:00:00',
               '2022-01-24 10:00:00', '2022-01-25 10:00:00',
               '2022-01-25 11:00:00', '2022-01-26 14:00:00',
               '2022-01-26 15

In [23]:
# 顯示每個站點缺失的時間點數量
for site_id, missing_count in missing_data_count_summary.items():
    print(f"站點 {site_id} 缺失的時間點數量：{missing_count}")

站點 25.0 缺失的時間點數量：125
站點 26.0 缺失的時間點數量：125
站點 27.0 缺失的時間點數量：125
站點 28.0 缺失的時間點數量：125
站點 29.0 缺失的時間點數量：131
站點 30.0 缺失的時間點數量：125
站點 31.0 缺失的時間點數量：125
站點 32.0 缺失的時間點數量：125
站點 33.0 缺失的時間點數量：125
站點 34.0 缺失的時間點數量：125
站點 35.0 缺失的時間點數量：125
站點 36.0 缺失的時間點數量：125
站點 37.0 缺失的時間點數量：145
站點 38.0 缺失的時間點數量：125
站點 41.0 缺失的時間點數量：125
站點 69.0 缺失的時間點數量：126
站點 72.0 缺失的時間點數量：125
站點 83.0 缺失的時間點數量：125


In [15]:
# 重設索引
final_df = final_df.reset_index(drop=True)

# 檢查填補後每個站點的缺失值
filled_missing_data_summary = {}
for site_id in site_ids:
    site_df = final_df[final_df['siteid'] == site_id].copy()
    site_df = site_df.sort_values(by='datacreationdate')
    expected_dates = pd.date_range(start=site_df['datacreationdate'].min(), end=site_df['datacreationdate'].max(), freq='h')
    missing_dates = expected_dates[~expected_dates.isin(site_df['datacreationdate'])]
    filled_missing_data_summary[site_id] = missing_dates

# 顯示每個站點填補後的缺失時間點
for site_id, missing_dates in filled_missing_data_summary.items():
    if not missing_dates.empty:
        print(f"填補後站點 {site_id} 仍然缺失的時間點：")
        print(missing_dates)
    else:
        print(f"填補後站點 {site_id} 沒有缺失的時間點。")


填補後站點 25.0 沒有缺失的時間點。
填補後站點 26.0 沒有缺失的時間點。
填補後站點 27.0 沒有缺失的時間點。
填補後站點 28.0 沒有缺失的時間點。
填補後站點 29.0 沒有缺失的時間點。
填補後站點 30.0 沒有缺失的時間點。
填補後站點 31.0 沒有缺失的時間點。
填補後站點 32.0 沒有缺失的時間點。
填補後站點 33.0 沒有缺失的時間點。
填補後站點 34.0 沒有缺失的時間點。
填補後站點 35.0 沒有缺失的時間點。
填補後站點 36.0 沒有缺失的時間點。
填補後站點 37.0 沒有缺失的時間點。
填補後站點 38.0 沒有缺失的時間點。
填補後站點 41.0 沒有缺失的時間點。
填補後站點 69.0 沒有缺失的時間點。
填補後站點 72.0 沒有缺失的時間點。
填補後站點 83.0 沒有缺失的時間點。


In [16]:
final_df.shape

(315360, 17)

In [17]:

# 選擇特定時間點進行打印
specific_missing_dates = pd.to_datetime([
    '2022-01-24 02:00:00', '2022-01-24 04:00:00',
    '2022-01-24 07:00:00', '2022-01-24 08:00:00'
])

for missing_date in specific_missing_dates:
    prev_hour = missing_date - pd.Timedelta(hours=1)
    data_at_missing = final_df[final_df['datacreationdate'] == missing_date]
    data_at_prev_hour = final_df[final_df['datacreationdate'] == prev_hour]
    
    print(f"缺失時間點: {missing_date}")
    print(data_at_missing)
    print(f"前一小時數據: {prev_hour}")
    print(data_at_prev_hour)
    print("\n")

缺失時間點: 2022-01-24 02:00:00
        siteid    aqi pollutant    status  so2    o3  o3_8hr   no2  windspeed  \
554       25.0   62.0     細懸浮微粒        普通  2.0  16.8     9.0  14.5        1.4   
18074     26.0   67.0     細懸浮微粒        普通  1.9   2.2     8.0  19.4        1.7   
35594     27.0   36.0       NaN        良好  1.7  10.5    17.0   8.5        2.7   
53114     28.0   79.0     細懸浮微粒        普通  1.4   0.6     4.0  18.3        2.1   
70634     29.0   66.0     細懸浮微粒        普通  0.6   7.0    10.0  13.5        3.0   
88154     30.0   93.0     細懸浮微粒        普通  1.0   0.4     5.0  23.6        1.1   
105674    31.0   85.0     細懸浮微粒        普通  1.2   0.5     3.0  28.8        1.1   
123194    32.0   87.0     細懸浮微粒        普通  1.3   1.3     6.0  24.3        1.6   
140714    33.0   96.0     細懸浮微粒        普通  2.7   9.7     8.0  16.5        1.9   
158234    34.0   70.0     細懸浮微粒        普通  1.9  11.9    15.0  12.2        3.0   
175754    35.0  108.0     細懸浮微粒  對敏感族群不健康  1.1  19.6    15.0   7.5        1.7   
1

In [18]:
# 保存 final_df 到 CSV 文件
final_df.to_csv('filled_time_data.csv', index=False)
